## Extra requirements
```bash
pip install -q -U google-genai
```

## Required environment variable
GEMINI_API_KEY

### How to get GEMINI API KEY

Use the link: [gemini api key](https://aistudio.google.com/app/apikey) to get yours.

In [12]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from google import genai
from google.genai import types

In [13]:
def gemini_invoke(website):
    load_dotenv()
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key or len(api_key) < 39:
        print("No correct api key was found")
        return
    else:
        print("Api key found. Good to go!")
        client = genai.Client(api_key=api_key)
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction=system_prompt),
                contents=user_prompt_for(website)
            )
        return response.text

In [14]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title was found"

        for irr in soup.body(["script", "style", "img", "input"]):
            irr.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
            

In [ ]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

In [15]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that navigation related. Respond \
in markdown."


In [16]:
messages = [{"role": "user", "content": system_prompt}]

In [17]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [20]:
def summarize(url, model="gemini"):
    website = Website(url)
    if model == "ollama":
        import ollama
        Model="llama3.2"
        messages[0]["content"] += f" Website: {url}"
        response = ollama.chat(model=Model, messages=messages)
        return response["message"]["content"]
    else:
        return gemini_invoke(website)

In [ ]:
summarize("https://edwarddonner.com", "ollama")